In [ ]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf

In [ ]:
#  Import and read the data
cannabis_df = pd.read_csv("Resources/DB INFO HERE")

In [ ]:
# Determine the number of unique values in each column.
cannabis_df.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
can_counts = cannabis_df.DATA_HERE.value_counts()

In [ ]:
#This is just a mockup. Much of this will change when we populate with actual data
# Determine which values to replace if counts are less than ...?
replace_can = list(can_counts[can_counts < 700].index)

# Replace in dataframe
for app in replace_can:
    can_df.DATA_TYPE = cannabis_df.DATA_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
cannabis_df.DATA_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
class_counts = cannabis_df.CLASSIFICATION.value_counts()

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_class = list(class_counts[class_counts < 700].index)

# Replace in dataframe
for cls in replace_class:
    cannabis_df.CLASSIFICATION = cannabis_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists
cannabis_cat = cannabis_df.dtypes[cannabis_df.dtypes == "object"].index.tolist()
# Check the number of unique values in each column
cannabis_df[cannabis_cat].nunique()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(cannabis_df[cannabis_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(cannabis_cat)
encode_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = cannabis_df.IS_SUCCESSFUL.values #Once again, Mockup. This will change
X = cannabis_df.drop(columns=["IS_SUCCESSFUL"]).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33, stratify=y)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the logistic regression model
log_classifier = LogisticRegression(solver="lbfgs",max_iter=200)

# Train the model
log_classifier.fit(X_train,y_train)

# Evaluate the model
y_pred = log_classifier.predict(X_test)
print(f" Logistic regression model accuracy: {accuracy_score(y_test,y_pred):.3f}")

In [ ]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128, random_state=33)

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest predictive accuracy: {accuracy_score(y_test,y_pred):.3f}")

In [ ]:
# Define the model - deep neural net
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 = 24
hidden_nodes_layer2 = 12

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Export our model to HDF5 file
nn.save("cannabis_data.h5")

In [ ]:
# Export data to a file
txt_file.write(cannabis_data.csv)